# Imports

In [1]:
from tlbo import TLBO
from dataclasses import dataclass, field
from typing import Union
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import multiprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, make_scorer, log_loss
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.optimizers import RMSprop
from scikeras.wrappers import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

2024-10-21 21:04:19.797028: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-21 21:04:19.798150: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 21:04:19.802451: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 21:04:19.815662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 21:04:19.836429: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

# RandomForest

In [2]:
# def run_model(model, X_train, y_train, X_test, y_test):

#     scores = cross_val_score(model, X_train, y_train, cv=3)
#     mean_score = scores.mean()

#     model.fit(X_train, y_train)

#     mean_score = round(mean_score, 4)

#     return scores, mean_score

# def evaluate_function(chromossome):

#     # data = pd.read_excel('/home/rafael/Mestrado/Artigo TLBO/database/Principal_homem1.xlsx', sheet_name='Homem 1')
#     data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem1.csv', delimiter=';')
#     label_counts = data['Output'].value_counts().sort_index()

#     # data = pd.read_csv('FinalDataset.csv')
#     # data = data.iloc[:, :16]

#     # label_counts = data['Label'].value_counts().sort_index()
#     data = data.dropna(axis=1)
#     colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]

#     data = data.drop(columns=colunas_com_infinitos)

#     y = data['Output']
#     # y = data['Label']
#     # X = data.drop(columns=['Label', 'Start Timestamp', 'End Timestamp'], inplace=True)
#     X = data.drop(columns=['Output'], inplace=True)
#     X = data

#     X = X.loc[:, np.array(chromossome, dtype=bool)]

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#     # oversampling = True

#     # if oversampling:
#     #     smote = SMOTE(random_state=42)
#     #     X_train, y_train = smote.fit_resample(X_train, y_train)

#         # print("Contagens após o oversampling:")
#         # print(pd.Series(y_train).value_counts().sort_index())

#     model_rf = RandomForestClassifier()

#     (scores_rf, 
#     mean_score_rf) = run_model(model=model_rf, 
#                                   X_train=X_train, 
#                                   y_train=y_train, 
#                                   X_test=X_test, 
#                                   y_test=y_test)
    
#     features = list(chromossome).count(1)
#     # return (1 - mean_score_rf) + 0.015*features
#     return (1 - mean_score_rf) + 0.001*features

In [2]:
def run_model(model, X_train, y_train, X_test, y_test):
    # Usando log_loss (ou outra métrica de erro)
    loss_scorer = make_scorer(log_loss, response_method="predict_proba")

    scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    mean_loss = scores.mean()

    model.fit(X_train, y_train)

    mean_loss = round(mean_loss, 4)

    return scores, mean_loss

def evaluate_function(chromossome):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    model_rf = RandomForestClassifier()

    (scores_rf, mean_loss_rf) = run_model(model=model_rf, 
                                          X_train=X_train, 
                                          y_train=y_train, 
                                          X_test=X_test, 
                                          y_test=y_test)
    
    features = list(chromossome).count(1)
    # A função agora retorna a perda média (log_loss) ajustada pela quantidade de features
    return mean_loss_rf + 0.001*features
    # return mean_loss_rf


In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            iterations_size=20)
 
best_student = tlbo.execute()

Learner Phase: 100%|██████████| 20/20 [00:00<00:00, 190217.87it/s]


Fitness: 0.7017| Best: [0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0
 1 1 0 0 0 0 1 1 0 1 0 1 1 1 0 0 0 1 1 0 1 0 1 0 1 1 0 0 1 1 0 0 1 1 0 1 0
 0 1 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 0 1 0 0 1 0 1 0 1 1 1 1
 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1
 0 0 1 0 1 0 0 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 1 1
 0 0 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 1 0 0 0 1 0 1 0 0 1
 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 0 0
 1 0 1 1 1 1 0 1 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 0 0 0 1 0 1 0
 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 1
 1 1 1 0 1 0 1 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0
 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0
 1

Learner Phase: 100%|██████████| 20/20 [00:00<00:00, 236966.33it/s]


Fitness: 0.698| Best: [0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0
 1 1 0 0 0 0 1 1 0 1 0 1 1 1 0 0 0 1 1 0 1 0 1 0 1 1 0 0 1 1 1 0 1 1 0 1 0
 0 1 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 0 1 0 1 0 0 1 1 1
 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1
 0 0 1 0 1 0 0 1 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 1 1
 0 0 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1
 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 1 0 0 0 1 0 1 0 0 1
 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 0 0
 1 0 1 1 1 1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1 0
 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 1
 1 1 1 0 1 0 1 1 0 1 0 1 1 0 0 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0
 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0
 1 

Learner Phase: 100%|██████████| 20/20 [00:00<00:00, 204102.38it/s]


Fitness: 0.7081999999999999| Best: [0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0
 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0
 0 1 1 0 0 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 1 0 0 1 0 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1
 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0
 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 0
 1 0 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0
 1 1 0 1 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0
 1 0 1 1 1 1 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0
 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 0 1 0 1 0 0 1
 1 1 1 0 0 0 1 1 0 0 1 1 1 1 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0
 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 

Calculating Population Fitness:   0%|          | 0/20 [00:09<?, ?it/s]


# SVM

In [2]:

def run_model(model, X_train, y_train, X_test, y_test):
    # Usando log_loss com probabilidades
    loss_scorer = make_scorer(log_loss, response_method="predict_proba")

    scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    mean_loss = scores.mean()

    model.fit(X_train, y_train)

    mean_loss = round(mean_loss, 4)

    return scores, mean_loss

def evaluate_function(chromossome):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    model_svm = SVC(probability=True)  # Ativando probabilidades para o SVM

    (scores_svm, mean_loss_svm) = run_model(model=model_svm, 
                                            X_train=X_train, 
                                            y_train=y_train, 
                                            X_test=X_test, 
                                            y_test=y_test)
    
    features = list(chromossome).count(1)
    
    return mean_loss_svm + 0.001*features


In [ ]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            iterations_size=20)
 
best_student = tlbo.execute()

# KNN

In [16]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher3.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # Usando o KNN com 5 vizinhos (padrão)
    model_knn = KNeighborsClassifier(n_neighbors=10)

    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_knn, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [17]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            iterations_size=20)
 
best_student = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [00:14<00:00,  1.37it/s]


------------------- Fitness: 1.986 | features: 320 -------------------

Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.20it/s]


------------------- Fitness: 1.973 | features: 368 -------------------

Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [00:15<00:00,  1.30it/s]


------------------- Fitness: 1.943 | features: 362 -------------------

Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [00:14<00:00,  1.37it/s]


------------------- Fitness: 1.943 | features: 362 -------------------

Accuracy in Training: 0.7729
Accuracy in Validation: 0.7898


np.float64(0.7729)

In [19]:
best_student.position


array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,

# MLP

## Homem 1

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            iterations_size=20)
 
best_student = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-10-21 20:18:25.286476: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Calculating Population Fitness: 100%|██████████| 20/20 [02:32<00:00,  7.63s/it]


------------------- Fitness: 0.885 | features: 298 -------------------

Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [02:31<00:00,  7.58s/it]


...

------------------- Fitness: 0.867 | features: 303 -------------------

Iteration: 15


Calculating Population Fitness: 100%|██████████| 20/20 [02:33<00:00,  7.67s/it]


------------------- Fitness: 0.867 | features: 303 -------------------

Iteration: 16


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]

: 

In [ ]:
best_student.position

## Homem 2

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem2.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

: 

In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            iterations_size=20)
 
best_student = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [03:18<00:00,  9.93s/it]


------------------- Fitness: 0.601 | features: 316 -------------------

Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [03:18<00:00,  9.94s/it]


------------------- Fitness: 0.577 | features: 316 -------------------

Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [03:24<00:00, 10.20s/it]


------------------- Fitness: 0.574 | features: 316 -------------------

Iteration: 13


Calculating Population Fitness: 100%|██████████| 20/20 [03:27<00:00, 10.38s/it]


------------------- Fitness: 0.574 | features: 316 -------------------

Iteration: 14


Calculating Population Fitness:  35%|███▌      | 7/20 [02:14<02:18, 10.68s/it] 

In [ ]:
best_student.position

## Mulher 1

In [ ]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [ ]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            iterations_size=20)
 
best_student = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

In [ ]:
best_student.position

## Mulher 2

In [ ]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher2.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [ ]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            iterations_size=20)
 
best_student = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

In [ ]:
best_student.position

## Mulher 3

In [ ]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher3.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [ ]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            iterations_size=20)
 
best_student = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

In [ ]:
best_student.position